In [2]:
import numpy as np
import pandas as pd
import scipy.optimize as opt
import yfinance as yf

In [24]:
# Define asset list
tickers = ['AAPL', 'MSFT', 'GS', 'AMZN', 'IBM']

In [25]:
data = yf.download(tickers, start='2010-01-01', end='2024-01-01')['Close']

[*********************100%***********************]  5 of 5 completed


In [26]:
data.head()

Ticker,AAPL,AMZN,GS,IBM,MSFT
Date,,,,,
2010-01-04,6.440331,6.6950,132.649918,74.269775,23.254047
2010-01-05,6.451466,6.7345,134.995087,73.372604,23.261557
2010-01-06,6.348847,6.6125,133.554276,72.895966,23.118809
2010-01-07,6.337109,6.5000,136.167725,72.643631,22.878380
2010-01-08,6.379241,6.6760,133.592636,73.372604,23.036161


In [27]:
# Compute daily log return
returns = np.log(data / data.shift(1)).dropna()

In [28]:
# Compute mean return
mu = returns.mean() * 252 # annualized mean return

In [29]:
# Compute covariance matrix
cov_matrix = returns.cov() * 252

In [30]:
# Define risk-free interest rate (e.g. US Treasury yield)
risk_free_rate = 0.03 

In [31]:
def sharpe_ratio(weights, mu, cov_matrix, risk_free_rate):
    portfolio_return = np.dot(weights, mu)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return -(portfolio_return - risk_free_rate) / portfolio_volatility # negative for minimization

In [32]:
# Set optimization constraints
num_assets = len(tickers)
initial_weights = np.ones(num_assets) / num_assets
bounds = [(0, 1) for _ in range(num_assets)]
constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]

In [33]:
opt_result = opt.minimize(sharpe_ratio, initial_weights, args=(mu, cov_matrix, risk_free_rate), method='SLSQP', bounds=bounds, constraints=constraints)

In [34]:
optimal_weights = opt_result.x

In [35]:
print('Optimal Portfolio Weights:')
for asset, weight in zip(tickers, optimal_weights):
    print(f'{asset}: {weight:.4f}')

Optimal Portfolio Weights:
AAPL: 0.5529
MSFT: 0.1891
GS: 0.0000
AMZN: 0.0000
IBM: 0.2579


In [36]:
final_sharpe = -sharpe_ratio(optimal_weights, mu, cov_matrix, risk_free_rate)
print(f'Optimal Sharpe Ratio: {final_sharpe:.4f}')

Optimal Sharpe Ratio: 0.8119
